# Definiendo una geogranja

En esta libreta crearemos una aplicacion que defina una region geografica y veremos como reaccionar ante eventos de localizacion: cuando el dispositivo salga de la region definida, la geogranja, se emite un mensaje y se registra que se salio de la region. Cuando entramos nuevamente a la region se notifica y se registra la entrada.

In [6]:
# import android
import android
import time
!~/.ipython/adBridge.sh

List of devices attached
09c3400a	device



In [7]:
droid = android.Android()
droid.ttsSpeak("listo para")

Result(id=0, result=None, error=None)

Vamos a revisar primero que el dispositivo esta "despierto":

In [5]:
droid.ttsSpeak("listo para")

ValueError: No JSON object could be decoded

In [3]:
droid.wakeLockAcquirePartial()

ValueError: No JSON object could be decoded

In [126]:
droidfile = '/sdcard/sl4a/RySI2018/locData.json'
locfile = 'data.json'
with open('data.json', 'a') as f:
    json.dump({}, outfile, indent=2)

In [200]:
%%file granja.py

import android
import time
from haversine import *

droid = android.Android()

lat1 = 19.5197093
lon1 = -96.9155696

droid.startLocating()
time.sleep(15) 
while True:     
    loc = droid.readLocation().result    
    if loc == {}:       
        loc = droid.getLastKnownLocation().result   
    if loc != {}:    
        try:
            n = loc['gps']    
        except KeyError:  
            n = loc['network']   
    la = n['latitude']  
    lo = n['longitude'] 
    
    print haversine(la, lo, lat1, lon1) 
    
    if haversine(la, lo, lat1, lon1) > .1:    
        droid.toggleRingerSilentMode(True)  
        droid.vibrate(1000)
        droid.ttsSpeak("saliendo de granja")
    else:         
        droid.toggleRingerSilentMode(False)

Overwriting granja.py


In [172]:
%%file haversine.py
# haversine function to get distance

from math import * 
 
def haversine(lon1, lat1, lon2, lat2):
    """     
    Calculate the great circle distance between two points      
    on the earth (specified in decimal degrees)     
    """     
    # convert decimal degrees to radians      
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])     
    # haversine formula      
    dlon = lon2 - lon1      
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2     
    c = 2 * atan2(sqrt(a), sqrt(1-a))      
    km = 6367 * c    
    return km  

Overwriting haversine.py


In [201]:
!adb push granja.py /sdcard/sl4a/scripts

[100%] /sdcard/sl4a/scripts/granja.py
granja.py: 1 file pushed. 0.2 MB/s (779 bytes in 0.003s)
